# 名企作业 2020-11-21, 石敏

## Action 1

## 针对Delicious数据集，对SimpleTagBased算法进行改进（使用NormTagBased、TagBased-TFIDF算法）
Delicious数据集：https://grouplens.org/datasets/hetrec-2011/

1867名用户，105000个书签，53388个标签

格式：
userID           bookmarkID            tagID                timestamp

In [1]:
import random
import math
import operator
import pandas as pd

In [2]:
file_path = "./delicious_2k/user_taggedbookmarks-timestamps.dat"
# 字典类型，保存了user对item的tag，即{userid: {item1:[tag1, tag2], ...}}
records = {}

In [3]:
# 数据加载
def load_data(path):
    print("开始数据加载...")
    df = pd.read_csv(path, sep='\t')
    for i in range(len(df)):
        uid = df['userID'][i]
        iid = df['bookmarkID'][i]
        tag = df['tagID'][i]
        # 键不存在时，设置默认值{}
        records.setdefault(uid,{})
        records[uid].setdefault(iid,[])
        records[uid][iid].append(tag)
    print("数据集大小为 %d." % (len(df)))
    print("设置tag的人数 %d." % (len(records)))
    print("数据加载完成\n")
    
load_data(file_path)


开始数据加载...
数据集大小为 437593.
设置tag的人数 1867.
数据加载完成



In [4]:
# 训练集，测试集
train_data = dict()
test_data = dict()

In [5]:
# 将数据集拆分为训练集和测试集
def train_test_split(ratio, seed=100):
    random.seed(seed)
    for u in records.keys():
        for i in records[u].keys():
            # ratio比例设置为测试集
            if random.random()<ratio:
                test_data.setdefault(u,{})
                test_data[u].setdefault(i,[])
                for t in records[u][i]:
                    test_data[u][i].append(t)
            else:
                train_data.setdefault(u,{})
                train_data[u].setdefault(i,[])
                for t in records[u][i]:
                    train_data[u][i].append(t)
                    
    # remove the users who are in test_data, but not in train_data
    for key in [k for k in test_data.keys() if k not in train_data]:
        test_data.pop(key, None)
    print("训练集样本数 %d, 测试集样本数 %d" % (len(train_data),len(test_data)))
    
# 训练集，测试集拆分，20%测试集
train_test_split(0.2)

训练集样本数 1860, 测试集样本数 1786


In [6]:
# 用户标签，商品标签
user_tags = dict()
tag_items = dict()
user_items = dict()
tag_users = dict()

In [7]:
# 使用dictionary统计值的个数,每有重复，值加1。
def count_weight(count_dict, key, value, weight=1):
    if key not in count_dict:
        count_dict.setdefault(key, {})
        count_dict[key].setdefault(value, weight)
    else:
        if value not in count_dict[key]:
            count_dict[key][value] = weight
        else:
            count_dict[key][value] += weight


# 使用训练集，初始化user_tags, tag_items, user_items, tag_users
def initStat():
    for user, bookmarks in train_data.items():
        for bookmark,tags in bookmarks.items():
            for tag in tags:
                #print tag
                # 用户和tag的关系
                count_weight(user_tags, user, tag)
                # tag和bookmark的关系
                count_weight(tag_items, tag, bookmark)
                # 用户和bookmark的关系
                count_weight(user_items, user, bookmark)
                # tag和用户的关系
                count_weight(tag_users, tag, user)
    print("user_tags, tag_items, user_items初始化完成.")
    print("user_tags: %d   user_items: %d" % (len(user_tags), len(user_items)))
    print("tag_users: %d,   tag_items: %d" % (len(tag_users), len(tag_items)))


initStat()

user_tags, tag_items, user_items初始化完成.
user_tags: 1860   user_items: 1860
tag_users: 36884,   tag_items: 36884


In [8]:
# 对用户user推荐Top-N
def recommend_simple_tag(user, N):
    simpleTag=dict()
    # 对Item进行打分，分数为所有的（用户对某标签使用的次数 wut, 乘以 商品被打上相同标签的次数 wti）之和
    for tag, wut in user_tags[user].items(): # 用户使用标签t的次数
        for item, wti in tag_items[tag].items(): #商品i被打过标签t的次数
            if item in user_items[user]: continue
            # 用户对该商品打过标签，就对该商品的SimpleTag进行累加
            if item not in simpleTag: simpleTag[item] = 0
            simpleTag[item] += wut * wti
    return sorted(simpleTag.items(), key=operator.itemgetter(1), reverse=True)[0:N]

# 使用测试集，计算准确率和召回率, 两率与Tag没有关系，只和商品有关系
def precisionAndRecall(N, recommend):
    hit, h_recall, h_precision = (0, 0, 0)
    for user,items in test_data.items():
        # 获取Top-N推荐列表
        rank = recommend(user, N)
        for item, _ in rank:
            if item in items:
                hit += 1        #相当于TP： 关注的商品在推荐列表里，算命中一次
        h_recall += len(items)  #TP + FN, 关注的商品总数
        h_precision += N        #TP + FP， 关注的商品在推荐列表里，不关注的也在推荐列表里，所以就是推荐表的长度
    #print('一共命中 %d 个, 一共推荐 %d 个, 用户设置tag总数 %d 个' %(hit, h_precision, h_recall))
    # 返回准确率 和 召回率
    return (hit/(h_precision*1.0)), (hit/(h_recall*1.0))


print("推荐结果评估(Simple Tag Based)：")
print("%3s %10s %10s" % ('N',"精确率",'召回率'))
for n in [5,10,20,40,60,80,100]:
    precision,recall = precisionAndRecall(n, recommend_simple_tag)
    print("%3d %10.3f%% %10.3f%%" % (n, precision * 100, recall * 100))


推荐结果评估(Simple Tag Based)：
  N        精确率        召回率
  5      0.829%      0.355%
 10      0.633%      0.542%
 20      0.512%      0.877%
 40      0.381%      1.304%
 60      0.318%      1.635%
 80      0.276%      1.893%
100      0.248%      2.124%


In [9]:
# 对用户user推荐Top-N
def recommend_norm_tag(user, N):
    simpleTag=dict()
    # 对Item进行打分，分数为所有的（用户对某标签使用的次数 wut, 乘以 商品被打上相同标签的次数 wti）之和
    for tag, wut in user_tags[user].items(): # 用户使用标签t的次数
        for item, wti in tag_items[tag].items(): #商品i被打过标签t的次数
            if item in user_items[user]: continue
            
            #norm
            norm = len(user_tags[user].items()) * len(tag_items[tag].items())                
            # 用户对该商品打过标签，就对该商品的SimpleTag进行累加
            if item not in simpleTag: simpleTag[item] = 0
            simpleTag[item] += wut * wti / norm
    return sorted(simpleTag.items(), key=operator.itemgetter(1), reverse=True)[0:N]

print("推荐结果评估(Norm Tag Based)：")
print("%3s %10s %10s" % ('N',"精确率",'召回率'))
for n in [5,10,20,40,60,80,100]:
    precision,recall = precisionAndRecall(n, recommend_norm_tag)
    print("%3d %10.3f%% %10.3f%%" % (n, precision * 100, recall * 100))

推荐结果评估(Norm Tag Based)：
  N        精确率        召回率
  5      0.806%      0.345%
 10      0.577%      0.494%
 20      0.428%      0.733%
 40      0.300%      1.026%
 60      0.259%      1.333%
 80      0.237%      1.620%
100      0.222%      1.903%


In [10]:
import math as m

# 对用户user推荐Top-N
def recommend_tfidf_tag(user, N):
    simpleTag=dict()
    # 对Item进行打分，分数为所有的（用户对某标签使用的次数 wut, 乘以 商品被打上相同标签的次数 wti）之和
    for tag, wut in user_tags[user].items(): # 用户使用标签t的次数
        for item, wti in tag_items[tag].items(): #商品i被打过标签t的次数
            if item in user_items[user]: continue
            
            #norm
            norm = m.log(1 + len(tag_users[tag].items()))                
            # 用户对该商品打过标签，就对该商品的SimpleTag进行累加
            if item not in simpleTag: simpleTag[item] = 0
            simpleTag[item] += wut * wti / norm
    return sorted(simpleTag.items(), key=operator.itemgetter(1), reverse=True)[0:N]

print("推荐结果评估(Tag Based - TFIDF)：")
print("%3s %10s %10s" % ('N',"精确率",'召回率'))
for n in [5,10,20,40,60,80,100]:
    precision,recall = precisionAndRecall(n, recommend_tfidf_tag)
    print("%3d %10.3f%% %10.3f%%" % (n, precision * 100, recall * 100))

推荐结果评估(Tag Based - TFIDF)：
  N        精确率        召回率
  5      1.008%      0.431%
 10      0.761%      0.652%
 20      0.549%      0.940%
 40      0.402%      1.376%
 60      0.328%      1.687%
 80      0.297%      2.033%
100      0.269%      2.306%
